# Problem Set \#4
## ECON 833, Computational Methods for Economists

This problem set has you estimate a model of the returns to wages via maximum likelihood.

The model itself is:
$$
ln(w_{i,t}) = \alpha + \beta_{1}Educ_{i,t} + \beta_{2}Age_{i,t} + \beta_{3}Age^2_{i,t} + \beta_{4}Black_{i,t} + \beta_{5}Hispanic_{i,t} + \beta_{6}OtherRace_{i,t} + \varepsilon_{i,t},
$$

where:
* $w_{i,t}$ = wage of individual $i$ in survey year $t$
* $Educ_{i,t}$ = education in years
* $Age_{i,t}$ = age in years
* $Black_{i,t}$, $Hispanic_{i,t}$, $OtherRace_{i,t}$ = dummy variables for race = Black, Hispanic, Not $\in$ \{White, Black, Hispanic\}.

Note that this model is linear, so we can check out maximum likelihood estimate against an OLS estimate of the same model (they should have the same point estimates).

The goal is to estimate this model for four years: 1971, 1980, 1990, 2000 and compare your estimates.


## Step 1: Read in PSID data and clean it up.

In [1]:
# imports
import pandas as pd
import numpy as np
import os
import scipy.optimize as opt
import scipy.stats as stats
import statsmodels.regression.linear_model as sm

In [2]:
# read in data - keeping only variabes of interest
data_file_path = os.path.join('..', 'Optimization', 'PS4_data.dta')
psid = pd.read_stata(data_file_path,
                     columns=['id68', 'year', 'hannhrs', 'hlabinc', 'hsex',
                              'hyrsed', 'age', 'hrace'])

# create wages and ln(wages)
# note need to be careful with wages = 0
psid['wage'] = psid['hlabinc']/psid['hannhrs']
psid['ln_wage'] = np.log(psid['wage'])

# create age squared
psid['age_sq'] = psid['age'] ** 2

# sample selection
psid.drop(psid[psid.hsex != 1].index, inplace=True)
psid.drop(psid[psid.age > 60].index, inplace=True)
psid.drop(psid[psid.age < 25].index, inplace=True)
psid.drop(psid[psid.wage < 7].index, inplace=True)
psid.drop(psid[psid.wage == np.inf].index, inplace=True)

# create dummy variables for race
psid['black'] = (psid['hrace'] == 2).astype(int)
psid['hispanic'] = (psid['hrace'] == 5).astype(int)
psid['other'] = (
    (psid['hrace'] == 3) | (psid['hrace'] == 4) |
    (psid['hrace'] == 6) | (psid['hrace'] == 7)).astype(int)

# drop obs if missing values for any variabls in regression model
psid.dropna(axis=0,
            subset=['ln_wage', 'hyrsed', 'age', 'age_sq', 'black', 'other'],
            inplace=True)

# add a constant
psid['const'] = 1

# put PSID dataframes for relevant year in a dict
year_list = [1971, 1980, 1990, 2000]
psid_dict = {}
for y in year_list:
    psid_dict[y] = psid[psid.year == y]

/Users/jason.debacker/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [3]:
# Look at data
psid.describe()

,id68,year,hannhrs,hlabinc,hsex,hyrsed,age,hrace,wage,ln_wage,age_sq,black,hispanic,other,const
count,57097.000000,57097.000000,57097.000000,5.709700e+04,57097.0,57097.000000,57097.000000,57062.000000,57097.000000,57097.000000,57097.000000,57097.000000,57097.0,57097.000000,57097.0
mean,1507.174860,1986.584129,2228.480713,5.282710e+04,1.0,13.529993,39.242939,1.101416,24.320503,3.010798,1631.272217,0.056343,0.0,0.022506,1.0
std,828.407534,8.716500,620.018860,5.235579e+04,0.0,2.449510,9.579581,0.369015,25.204367,0.544119,791.988159,0.230584,0.0,0.148322,0.0
min,1.000000,1971.000000,2.000000,1.666980e+01,1.0,1.000000,25.000000,1.000000,7.000252,1.945946,625.000000,0.000000,0.0,0.000000,1.0
25%,782.000000,1979.000000,1952.000000,3.037345e+04,1.0,12.000000,31.000000,1.000000,13.950494,2.635515,961.000000,0.000000,0.0,0.000000,1.0
50%,1542.000000,1987.000000,2160.000000,4.381145e+04,1.0,13.000000,38.000000,1.000000,19.914677,2.991457,1444.000000,0.000000,0.0,0.000000,1.0
75%,2225.000000,1994.000000,2519.000000,6.138394e+04,1.0,16.000000,47.000000,1.000000,27.793240,3.324793,2209.000000,0.000000,0.0,0.000000,1.0
max,2930.000000,2002.000000,5840.000000,3.771521e+06,1.0,17.000000,60.000000,3.000000,1717.330322,7.448526,3600.000000,1.000000,0.0,1.000000,1.0


## Maximum Likelihood Estimation

Log wages are modeled as:
$$
ln(w_{i,t}) = \alpha + \beta_{1}Educ_{i,t} + \beta_{2}Age_{i,t} + \beta_{3}Age^2_{i,t} + \beta_{4}Black_{i,t} + \beta_{5}Hispanic_{i,t} + \beta_{6}OtherRace_{i,t} + \varepsilon_{i,t},
$$

With the assumption that $\varepsilon_{i,t} \sim N(0, \sigma^2)$.

In MLE, we want to find the parameters, $\hat{\beta}$ and $\hat{\sigma}$, that maximize the likeihood of observing the outcome (log wages) conditional on the data.  i.e., for each year (which you will do separately for this question) we want to solve:

$$
max_{\hat{\beta}, \hat{\sigma}}\prod_{i=1}^{N}p(y_{i,t}|x_{i};\hat{\beta}, \hat{\sigma})
$$

With the assumption that $\varepsilon$ is distributed normally with mean zero and variance $\sigma^2$, we know the functional form of the conditional probability function:

$$
\prod_{i=1}^{N}\frac{1}{\sqrt{2\pi \sigma^2}} e^{-\frac{\left(y_{i} - \hat{\beta} x_{i}\right)^2}{2 \hat{\sigma}^2}}
$$

We can estimate this, but it's often simpler to deal with the log-likelihood function:

$$
LL = log \left( \prod_{i=1}^{N}\frac{1}{\sqrt{2\pi \hat{\sigma}^2}} e^{-\frac{\left(y_{i} - \hat{\beta} x_{i}\right)^2}{2 \hat{\sigma}^2}} \right)
$$

$$
LL =  \prod_{i=1}^{N}log \left(\frac{1}{\sqrt{2\pi \hat{\sigma}^2}} e^{-\frac{\left(y_{i} - \hat{\beta} x_{i}\right)^2}{2 \hat{\sigma}^2}}\right)
$$

$$
LL = -\frac{N}{2}log(2\pi) - N log(\hat{\sigma}) - \frac{1}{2\hat{\sigma}^2}\sum_{i=1}^{N}\left( y_{i} - \hat{\beta} x_{i}\right)^2
$$

It's this LL function that we will use to estimate $\beta$, $\sigma$.

### Step 1: Define the statistical objective function

The first thing we'll do then is to write out this statistical objective function that is the LL function.


In [4]:
def ll_func(params, data):
    '''
    The log-likelihood function.
    
    Args:
        params (tuple): model parameters
        data (Pandas DataFrame): data, contains covariates in model
    
    Returns:
        loglike (scalar): the negative value of the log likelihood function
            We'll use a negative value so we can use a minimizer to max the LL
    '''
    # unpack tuple of parameters
    alpha, beta1, beta2, beta3, beta4, beta5, sigma = params

    # find number of obs
    n = data['const'].count()
    
    # compute predicted value for convenience
    y_hat = (
        alpha * data['const'] + beta1 * data['hyrsed'] +
        beta2 * data['age'] + beta3 * data['age_sq'] +
        beta4 * data['black'] + beta5 * data['other'])
    
    # compute log likelihood value
    # By hand:
#     loglike = (
#         -1 * (n / 2) * np.log(2 * np.pi) - n * np.log(sigma) -
#         (1 / (2 * sigma ** 2)) * ((data['ln_wage'] - y_hat) ** 2).sum())
    # using Scipy stats functions:
    loglike = stats.norm(y_hat, sigma).logpdf(data['ln_wage']).sum()
  
    return -loglike      

### Step 2: Estimate the model

Now we call the LL function with an optimizer to estimate the parameters.

We do have some bounds on $\sigma$, so we'll use a bounded optimizer, constraining $\sigma > 0$.

We do this separately for each year.

In [5]:
# set initial guesses
Theta0 = (0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.5)

# set bounds on parameters
# only relevant bound is that sigma > 0
bnds = ((None, None), (None, None), (None, None),
        (None, None), (None, None), (None, None),
        (1e-12, None))

# Minimize log-likelihood function for model of each year
ll_results = {}   # empty dict to put results in
for y in year_list:
    # do it once
    res = opt.minimize(
        ll_func, Theta0, args=(psid_dict[y],),
        method="L-BFGS-B", bounds=bnds, tol=1e-15)
    # then again with better starting values
    # we use results from prior to get starting values
    ll_results[y] = opt.minimize(
        ll_func, res['x'], args=(psid_dict[y],),
        method="Nelder-Mead", tol=1e-15)

### Step 4: Format results

In [6]:
# Put all results together in a dictionary
results_dict = {'Variable': ['Constant', 'Education', 'Age', r'$Age^2$', 'Black', 'Other']}
for y in year_list:
    results_dict[str(y)] = ll_results[y]['x'][:-1]
# then to dataframe for nice output (can save to csv, txt, latex, etc from here)
df = pd.DataFrame(results_dict)
df

,Variable,1971,1980,1990,2000
0,Constant,0.586047,1.002579,0.277367,-0.293170
1,Education,0.066498,0.066014,0.095493,0.110332
2,Age,0.064918,0.045550,0.057863,0.084355
3,$Age^2$,-0.000617,-0.000399,-0.000540,-0.000887
4,Black,-0.164127,-0.103215,-0.168033,-0.259585
5,Other,0.017595,0.012271,-0.051991,-0.062037


## Checking our work

Here, we compare the MLE results estimates obtained from OLS.

We use the `statsmodels` `OLS` module to obtain these estimates.

In [7]:
ols_results = {}  # empty dict to put results in
for y in year_list:
    # initialize regression model
    reg = sm.OLS(
        endog=psid_dict[y]['ln_wage'],
        exog=psid_dict[y][['const', 'hyrsed', 'age', 'age_sq','black', 'other']],
        missing='drop')
    # estimate and put results object in dict
    ols_results[y] = reg.fit()

Now we have our OLS results, we'll just format and display similar to what we did for the MLE:

In [8]:
# Put all results together in a dictionary
ols_results_dict = {'Variable': ['Constant', 'Education', 'Age', r'$Age^2$', 'Black', 'Other']}
for y in year_list:
    ols_results_dict[str(y)] = ols_results[y].params.values
# then to dataframe for nice output (can save to csv, txt, latex, etc from here)
ols_df = pd.DataFrame(ols_results_dict)
ols_df

,Variable,1971,1980,1990,2000
0,Constant,0.586331,1.002272,0.277243,-0.293275
1,Education,0.066500,0.066003,0.095499,0.110333
2,Age,0.064904,0.045569,0.057864,0.084360
3,$Age^2$,-0.000617,-0.000399,-0.000540,-0.000887
4,Black,-0.164137,-0.103028,-0.167988,-0.259617
5,Other,0.017512,0.012315,-0.052001,-0.062144


Now our test that these coefficients are all the same -- just difference the DataFrames:

In [9]:
# We can't difference strings, so lets
# make the one column with string variables
# into an index
df = df.set_index('Variable')
ols_df = ols_df.set_index('Variable')
# now we difference
df - ols_df

,1971,1980,1990,2000
Variable,,,,
Constant,-2.837366e-04,3.070889e-04,1.236912e-04,1.047856e-04
Education,-1.784841e-06,1.197877e-05,-6.114836e-06,-7.178687e-07
Age,1.349101e-05,-1.825917e-05,-1.060251e-06,-5.636919e-06
$Age^2$,-1.464605e-07,1.595718e-07,-1.936842e-08,7.554676e-08
Black,9.975204e-06,-1.866130e-04,-4.593685e-05,3.273829e-05
Other,8.327829e-05,-4.346030e-05,9.494789e-06,1.071488e-04


## Bonus, estimate OLS with `numpy`

Here, we use array operations from `numpy` to produce our OLS estimates.

We then verify those are identical to what is returned from `sm.OLS`.

In [10]:
# Put all results together in a dictionary
matrix_results_dict = {'Variable': ['Constant', 'Education', 'Age', r'$Age^2$', 'Black', 'Other']}
for y in year_list:
    X = np.array(psid_dict[y][['const', 'hyrsed', 'age', 'age_sq', 'black', 'other']])
    Y = np.array(psid_dict[y][['ln_wage']])
    ols_mat = np.dot(np.linalg.inv(np.dot(np.transpose(X),X)), np.dot(np.transpose(X),Y))
    matrix_results_dict[str(y)] = ols_mat.squeeze()
# then to dataframe for nice output (can save to csv, txt, latex, etc from here)
matrix_df = pd.DataFrame(matrix_results_dict)
matrix_df

,Variable,1971,1980,1990,2000
0,Constant,0.586331,1.002272,0.277243,-0.293275
1,Education,0.066500,0.066003,0.095499,0.110333
2,Age,0.064904,0.045569,0.057864,0.084360
3,$Age^2$,-0.000617,-0.000399,-0.000540,-0.000887
4,Black,-0.164137,-0.103028,-0.167988,-0.259617
5,Other,0.017512,0.012315,-0.052001,-0.062144


In [11]:
# Show diffs in results
matrix_df = matrix_df.set_index('Variable')
matrix_df - ols_df

,1971,1980,1990,2000
Variable,,,,
Constant,-1.235678e-12,2.625899e-12,-3.387290e-12,5.762391e-12
Education,7.245593e-14,-1.363909e-13,2.526451e-13,-3.941014e-13
Age,1.962319e-14,-9.346690e-15,-1.221245e-15,-2.761680e-15
$Age^2$,-4.531965e-17,-5.789640e-17,-9.746978e-17,2.475234e-16
Black,-3.330669e-16,-1.068590e-15,4.468648e-15,-2.997602e-15
Other,1.443290e-15,-2.636780e-16,1.720846e-15,-2.081668e-16
